In [ ]:
"""


當用戶關注Line@後，Line會發一個FollowEvent，

我們接受到之後，取得用戶個資，對用戶綁定自定義菜單，會回傳四個消息給用戶


"""

In [5]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節 (搭建一個空的伺服器)
app = Flask(__name__,static_url_path = "/food" , static_folder = "../food/")

# 生成實體物件 
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
# 稽核碼
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        # 把消息內容跟簽章交給 handler 做檢查跟後續業務的處理
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


In [6]:
'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
TextSendMessage(text="歡迎光臨無人商店"),
    TextSendMessage(text="更多資訊請點選下列選單")
]

In [7]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("../foodusers.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [8]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.3 - - [23/Jan/2019 14:36:07] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:08] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:13] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:13] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:14] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:21] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:22] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:22] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:23] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:24] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:29] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:30] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:30] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:31] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [23/Jan/2019 14:36:32] "POST / HTTP/1.1" 200 -
